# GLTF 格式教學 Accessor 篇

<a href="https://colab.research.google.com/github/CSP-GD/notes/blob/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/accessor/accessor.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

[`Open in observablehq`](https://observablehq.com/@toonnyy8/gltf-accessor)

![圖 1. buffers, bufferViews, accessors \[1\]](https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/accessor/gltfOverview-2.0.0b-accessor.png)

圖 1. buffers, bufferViews, accessors \[1\]

## 簡介

在 glTF，模型的網格、權重、動畫等等數據實際上是儲存在 Buffer 中，  
當要使用到這些數據時，就會用到 Accessor 去解讀數據，  
而 Accessor 解讀的數據則是透過 BufferView 去從 Buffer 中提取出來的。

運作流程如下  
> **Buffer** ==> **BufferView** 提取數據 ==> **Accessor** 解讀數據 ==> 數據

### Accessor 屬性

- bufferView : \<`number`\>  
> 此 Accessor 是從哪個 BufferView 取得數據。

- byteOffset :\<`number`\>  
> 從 BufferView 偏移多少個 byteOffset 的位置開始取數據。

- type : <`string`>  
> 表示一筆數據的類型(count 的單位)  
> `SCALAR` : $1$ 個 componentType 構成  
> `VEC2` : $2$ 個 componentType 構成  
> `VEC3` : $3$ 個 componentType 構成  
> `VEC4` : $4$ 個 componentType 構成  
> `MAT2` : $2*2$ 個 componentType 構成  
> `MAT3` : $3*3$ 個 componentType 構成  
> `MAT4` : $4*4$ 個 componentType 構成  

- componentType : \<`GL Constant of Data Type`\>  
> 表示數據的型別，以下幾種為部分 componentType 代表的型別  
> `5120` : `BYTE`  
> `5121` : `UNSIGNED_BYTE`  
> `5122` : `SHORT`  
> `5123` : `UNSIGNED_SHORT`  
> `5124` : `INT`  
> `5125` : `UNSIGNED_INT`  
> `5126` : `FLOAT`  
  
- count : \<`number`\>  
> 有幾筆數據

- min : \<`type<componentType>`\>
> 數據的最大值

- max : \<`type<componentType>`\>
> 數據的最小值


### BufferView 屬性

- buffer : \<`number`\>  
> 此 BufferView 是從哪個 Buffer 取得數據。

- byteOffset : \<`number`\>  
> 從 Buffer 偏移多少個 byteOffset 的位置開始取數據。

- byteLength : \<`number`\>  
> 要取下多少個 byte。

- byteStride : \<`number`\>  
> 數據交錯擺放時，讓 Accessor 知道取數據的步伐要多少。

- target : \<`34962`|`34963`\>  
> 用來分辨數據的性質為 vertex (target 等於 `34962`，代表 `ARRAY_BUFFER`) 還是 vertex indices (target 等於 `34963`，代表 `ELEMENT_ARRAY_BUFFER`)。

### Buffer 屬性

- byteLength : \<`number`\>  
> 此 Buffer 的大小。

- uri : \<`string`\>  
> bufferData 的位置，也可能用 base64 直接儲存 bufferData。

## 正式開始

### 載入 glTF_tools

In [20]:
!wget https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/gltf-tools.ipynb -O gltf-tools.ipynb
%run ./gltf-tools.ipynb

--2020-05-02 03:11:29--  https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/gltf-tools.ipynb
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/gltf-tools.ipynb [following]
--2020-05-02 03:11:29--  https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/gltf-tools.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6682 (6.5K) [text/plain]
Saving to: ‘gltf-tools.ipynb’

gltf-tools.ipynb    100%[==============

## 載入檔案

In [21]:
!wget https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/accessor/cube.glb -O cube.glb
glb_file = open('./cube.glb', 'rb')
glb_bytes = glb_file.read()
model, buffers = glTF_tools.glb_loader(glb_bytes)

--2020-05-02 03:11:32--  https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/accessor/cube.glb
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/accessor/cube.glb [following]
--2020-05-02 03:11:32--  https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/accessor/cube.glb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1916 (1.9K) [application/octet-stream]
Saving to: ‘cube.glb’

cube.glb            100%[=====

In [24]:
glTF_tools.render_JSON(model)

## 查看一下 Accessor 

In [0]:
#@title 發現少了 bufferOffset 的預設值 { vertical-output: true }
glTF_tools.render_JSON(model['accessors'])

### 因此我們在這邊製作一個為 Accessor 填入初始值的 function

In [0]:
import copy
def accessor_norm(accessor):
    _accessor = copy.deepcopy(accessor)

    # 做一些簡單的檢查，以確認必要的屬性是否存在
    if _accessor.get("bufferView") == None:
        raise AttributeError("The accessor does not have the attribute of bufferView")
    if _accessor.get("componentType") == None:
        raise AttributeError("The accessor does not have the attribute of componentType")
    if _accessor.get("count") == None:
        raise AttributeError("The accessor does not have the attribute of count")
    if _accessor.get("type") == None:
        raise AttributeError("The accessor does not have the attribute of type")

    buffer_offset = _accessor.get("bufferOffset")
    _accessor["bufferOffset"] = buffer_offset if buffer_offset != None else 0

    return _accessor

將 Accessor 處理過後，確認 bufferOffset 的屬性都有添加上去了

In [29]:
_model = copy.deepcopy(model)
for idx, accessor in enumerate(_model['accessors']):
    _model['accessors'][idx] = accessor_norm(accessor)

glTF_tools.render_JSON(_model['accessors'])

## 查看一下 BufferView

In [0]:
#@title 發現 byteStride 與 target 都沒有設定預設值 { vertical-output: true }
glTF_tools.render_JSON(_model['bufferViews'])

### 因此我們在這邊製作一個為 BufferView 填入初始值的 function

In [0]:
import copy
def buffer_view_norm(buffer_view, target = 34962):
    _buffer_view = copy.deepcopy(buffer_view)

    # 做一些簡單的檢查，以確認必要的屬性是否存在
    if _buffer_view.get("buffer") == None:
        raise AttributeError("The bufferView does not have the attribute of buffer")
    if _buffer_view.get("byteLength") == None:
        raise AttributeError("The bufferView does not have the attribute of byteLength")
    if _buffer_view.get("byteOffset") == None:
        raise AttributeError("The bufferView does not have the attribute of byteOffset")

    byte_stride = _buffer_view.get("byteStride")
    _buffer_view["byteStride"] = byte_stride if byte_stride != None else 0

    _target = _buffer_view.get("target")
    _buffer_view["target"] = _target if _target != None else target

    return _buffer_view

BufferView 的處理比 Accessor 複雜一些，  
因為 target 的數值要從 Mesh 中判斷，  
如果是被 Mesh 的 indices 指向的 Accessor 所標定的 BufferView，  
其 target 便是 ELEMENT_ARRAY_BUFFER（34963），  
否則就是 ARRAY_BUFFER（34962）。

In [0]:
_model = copy.deepcopy(model)
for idx, accessor in enumerate(_model['accessors']):
    _model['accessors'][idx] = accessor_norm(accessor)

glTF_tools.render_JSON(_model['accessors'])

In [0]:
def accessor_with_buffer(idx, model, buffers):
    _accessor = model['accessors'][idx]
    _buffer_view = model['bufferViews'][_accessor['bufferView']]
    _buffer = buffers[_buffer_view['buffer']]
    byteLength = _buffer_view['byteLength']
    byteOffset = _buffer_view['byteOffset']
    
    return _accessor, _buffer[byteOffset:byteOffset + byteLength]

accessor_with_buffer(0, model, buffers)

({'bufferView': 0,
  'componentType': 5126,
  'count': 24,
  'max': [1, 1, 1],
  'min': [-1, -1, -1],
  'type': 'VEC3'},
 b'\x00\x00\x80?\x00\x00\x80?\x00\x00\x80?\x00\x00\x80?\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80?\x00\x00\x80?\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80\xbf\x00\x00\x80\xbf\x00\x00\x80\xbf\x00\x00\x80\xbf\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80\xbf\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80?\x00\x00\x80?\x00\x00\x80?\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80\xbf\x00\x00\x80?\x00\x00\x80?\x00\x00\x80\xbf\x00\x00\x80

## 參考

1. https://github.com/KhronosGroup/glTF